In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
if not os.path.exists('./certs'):
    os.mkdir('certs')

In [3]:
os.listdir()

['.idea',
 '.ipynb_checkpoints',
 'ALX-T Connect Weekly Schedule - Data.pdf',
 'certs',
 'cleaning_and_splitting_movie_genres.ipynb',
 'connect_sessions_studyguide.pdf',
 'data-wrangling-template.ipynb',
 'datasets',
 'data_visualization.ipynb',
 'data_wrangling.ipynb',
 'downloadng_ebert_images.ipynb',
 'ebert_imgs.csv',
 'extract_attendance_from_cloudSQL.ipynb',
 'img',
 'img_errors.json',
 'lesson3_the_data_analysis_process.ipynb',
 'load_students_table_cloudsql.ipynb',
 'media_wiki.ipynb',
 'merging-data.ipynb',
 'name.png',
 'testing.ipynb',
 'web_scraping.ipynb',
 'wine_visualizations_solutions.ipynb']

In [4]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [5]:
config = {
    'user': 'root',
    'password': 'roots',
    'host': '34.133.227.55',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'certs/server-ca.pem',
    'ssl_cert': 'certs/client-cert.pem',
    'ssl_key': 'certs/client-key.pem'
}

In [6]:
# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [7]:
if cnxn.is_connected():
    # Get connection info
    db_Info = cnxn.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    # Initiaize a cursor object
    cursor = cnxn.cursor()

Connected to MySQL Server version  8.0.26-google


In [8]:
if False:
    # create a new 'alxt' database
    cursor.execute('CREATE DATABASE IF NOT EXISTS alxt')
    # select database
    cursor.execute('USE alxt')
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)
    cnxn.close()  # close connection because we will be reconnecting to alxt
    config['database'] = 'alxt'  # add new database to config dict
    cnxn = mysql.connector.connect(**config)
    cursor = cnxn.cursor()

In [9]:
cursor.execute('USE alxt')  # Use the alxt database

In [10]:
cursor.execute("SHOW TABLES;")
out = cursor.fetchall()
out

[('att_07_09',)]

In [12]:
df = pd.read_csv('datasets/session-4356-report-6_21_2022.csv').loc[:,['First Name', 'Last Name', 'Email']]
df.shape

(54, 3)

In [13]:
df.head(3)

,First Name,Last Name,Email
0,kelvin,mwaka,kelvinmwaka@gmail.com
1,Etuonu,David,davidetuonu12@gmail.com
2,Emmanuel,Otiti,fammava@gmail.com


In [14]:
df['status'] = np.repeat(0, len(df))
df.head(3)

,First Name,Last Name,Email,status
0,kelvin,mwaka,kelvinmwaka@gmail.com,0
1,Etuonu,David,davidetuonu12@gmail.com,0
2,Emmanuel,Otiti,fammava@gmail.com,0


In [16]:
df.columns = [col.lower().replace(' ','_').strip() for col in df.columns]
df.head(3)

,first_name,last_name,email,status
0,kelvin,mwaka,kelvinmwaka@gmail.com,0
1,Etuonu,David,davidetuonu12@gmail.com,0
2,Emmanuel,Otiti,fammava@gmail.com,0


In [18]:
cols = list(df.columns[:2])

for col in cols:
    df[col] = df[col].str.capitalize()
df.head(3)

,first_name,last_name,email,status
0,Kelvin,Mwaka,kelvinmwaka@gmail.com,0
1,Etuonu,David,davidetuonu12@gmail.com,0
2,Emmanuel,Otiti,fammava@gmail.com,0


In [20]:
df['email'] = df['email'].str.lower()

In [21]:
# create a students table

cursor.execute("CREATE TABLE IF NOT EXISTS students ("
               "first_name VARCHAR(100),"
               "last_name VARCHAR(100),"
               "email VARCHAR(100),"
               "status INT(1) )")

cnxn.commit()  # this commits changes to the database

In [22]:
# Lets see the table
cursor.execute('SHOW TABLES;')
out = cursor.fetchall()
out

[('att_07_09',), ('students',)]

In [23]:
def df_to_table(df):
    """This method loads a dataframe
        into a given table
    """
    for ind, row in df.iterrows():
        first, last = row.first_name, row.last_name
        email, status = row.email, row.status
        cursor.execute(f'INSERT INTO students (first_name, last_name, email, status) \
        VALUES ("{first}", "{last}", "{email}", "{status}")')
        
    cnxn.commit()  # this commits changes to the database

In [28]:
if False:
    df_to_table(df)

In [29]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 20) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [30]:
query = 'SELECT * FROM students;'
query_to_df(query)

Query ran for 0.3002662658691406 secs!


,first_name,last_name,email,status
0,Kelvin,Mwaka,kelvinmwaka@gmail.com,0
1,Etuonu,David,davidetuonu12@gmail.com,0
2,Emmanuel,Otiti,fammava@gmail.com,0
3,Chimezie,Ogbuu,ogbuchimezie17@gmail.com,0
4,Mugendi,Muthimi,mugendimike80@gmail.com,0
...,...,...,...,...
49,Moses,Bernard,mosesbernard5@gmail.com,0
50,Samuel,Sofela,sofela.olufemi@gmail.com,0
51,Stephanie,Elluh,elluhstephanie@gmail.com,0
52,Julius,Murigi,murigijulius1@gmail.com,0


In [31]:
cursor.close()

True